In [1]:
#@title Get text train and test ,X and Y
nrows_train = 20000 # @param {type:"integer"}
sequence_length = 13 # @param {type:"integer"}

feed_lenght = 8

import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import json
from keras.preprocessing.text import tokenizer_from_json

#load fusion
fus = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LECO/LECOfusion.csv",
                  parse_dates=["time"], index_col="time")

#Get the train and test train
texts_train = fus["fusion"].sample(nrows_train,)
texts_test = fus["fusion"].drop(texts_train.index)

#save texts test
texts_test.to_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LECO/LECOtexts_test.csv")

# Tokenize text
tokenizer = Tokenizer()

#tokenizer.fit_on_texts(texts_train)
tokenizer.fit_on_texts(fus["fusion"])

#Save tokenizer
tokenizer_json = tokenizer.to_json()

# Save the JSON configuration to a file
with open('/content/drive/MyDrive/Colab Notebooks/gpt/LECO/LECOtokenizer_config.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

sequences = tokenizer.texts_to_sequences(texts_train)

# Prepare input and output data
X = []
y = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        x_seq = sequence[:i]
        x_seq_padded = pad_sequences([x_seq], maxlen=sequence_length, padding='pre')
        X.append(x_seq_padded[0])
        y.append(sequence[i])

X = np.array(X)
y = np.array(y)

#filter seed/model words
df = pd.DataFrame(X)
df["y"] = y
df_fil =  df[(df.iloc[:, :sequence_length-feed_lenght+1] != 0).any(axis=1)]
X = df_fil.iloc[:, :-1].values
y = df_fil.iloc[:, -1].values

# One hot encode the outputs
y = np.eye(len(tokenizer.word_index) + 1)[y]

print("Dictionary size (words): len(y[i]):",len(y[1]) )
print("Total X variables:",len(X) )

print("\nText train example1")
display(texts_train[0])

print("\nSequences example 1")
display(sequences[0])

print("\nText train example2")
display(texts_train[1])

print("\nSequences example 2")
display(sequences[1])

print("\nX")
display(X[:25])

print("\nDictionary first 10 words")
display({k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]})


Dictionary size (words): len(y[i]): 3333
Total X variables: 163928

Text train example1


'16003kt 9999 prec0n CL2 CM0 11 10 q1024 18003KT 9999 NCD 10/10 Q1024'


Sequences example 1


[89, 1, 2, 45, 3, 9, 11, 38, 93, 1, 98, 11, 11, 38]


Text train example2


...

[526,
 34,
 20,
 48,
 49,
 6,
 14,
 135,
 2710,
 579,
 1,
 139,
 92,
 8,
 14,
 95,
 17,
 50,
 1116]


X


array([[   0,    0,    0,    0,    0,   89,    1,    2,   45,    3,    9,
          11,   38],
       [   0,    0,    0,    0,   89,    1,    2,   45,    3,    9,   11,
          38,   93],
       [   0,    0,    0,   89,    1,    2,   45,    3,    9,   11,   38,
          93,    1],
       [   0,    0,   89,    1,    2,   45,    3,    9,   11,   38,   93,
           1,   98],
       [   0,   89,    1,    2,   45,    3,    9,   11,   38,   93,    1,
          98,   11],
       [  89,    1,    2,   45,    3,    9,   11,   38,   93,    1,   98,
          11,   11],
       [   0,    0,    0,    0,    0,  526,   34,   20,   48,   49,    6,
          14,  135],
       [   0,    0,    0,    0,  526,   34,   20,   48,   49,    6,   14,
         135, 2710],
       [   0,    0,    0,  526,   34,   20,   48,   49,    6,   14,  135,
        2710,  579],
       [   0,    0,  526,   34,   20,   48,   49,    6,   14,  135, 2710,
         579,    1],
       [   0,  526,   34,   20,   48,   49,    6, 


Dictionary first 10 words


{'9999': 1,
 'prec0n': 2,
 'cm0': 3,
 'nosig': 4,
 'cl0': 5,
 '12': 6,
 '13': 7,
 '14': 8,
 '11': 9,
 '15': 10}

In [2]:
#@title Train the model

from keras.layers import Bidirectional

# Build the LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=20, input_length=sequence_length),
    Bidirectional(LSTM(130)),

    Dense(len(tokenizer.word_index)+1, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50,batch_size=512)

#save the model
model.save("/content/drive/MyDrive/Colab Notebooks/gpt/LECO/LECOmodel.keras")


Epoch 1/50
321/321 [==============================] - 25s 69ms/step - loss: 4.1228 - accuracy: 0.1946
Epoch 2/50
321/321 [==============================] - 22s 69ms/step - loss: 2.8905 - accuracy: 0.2890
Epoch 3/50
321/321 [==============================] - 22s 68ms/step - loss: 2.5717 - accuracy: 0.3303
Epoch 4/50
321/321 [==============================] - 22s 68ms/step - loss: 2.4171 - accuracy: 0.3550
Epoch 5/50
321/321 [==============================] - 22s 68ms/step - loss: 2.3192 - accuracy: 0.3707
Epoch 6/50
321/321 [==============================] - 22s 68ms/step - loss: 2.2473 - accuracy: 0.3815
Epoch 7/50
321/321 [==============================] - 22s 69ms/step - loss: 2.1799 - accuracy: 0.3928
Epoch 8/50
321/321 [==============================] - 22s 69ms/step - loss: 2.1228 - accuracy: 0.4024
Epoch 9/50
321/321 [==============================] - 22s 68ms/step - loss: 2.0654 - accuracy: 0.4138
Epoch 10/50
321/321 [==============================] - 22s 68ms/step - loss: 2.010